# IAR4 Exercise

**Collision Avoidance:** If the robot detects an obstacle with its front horizontal proximity sensors  (i.e., $\texttt{robot['prox.horizontal'][0] > 1000}, \texttt{robot['prox.horizontal'][1] > 1000}, ...)$, it turns clockwise until the obstacle is not detected anymore. The turning direction is fixed to avoid thrashing. 
 
**Random Walk:** The robot alternates between driving straight forward for $5~\textrm{s}$ and turning in a random direction for~$1~\textrm{s}$. 
  
**Line Following:** If the robot detects the line on the ground with both ground IR sensors, it drives straight forward. 
 If it detects the line only with its right sensor, it turns left. 
 If it detects the line only with its left sensor, it turns right.


In [1]:
import time
from loop_rate_limiters import RateLimiter
import random

# import thymiodirect package
from thymiodirect import Connection 
from thymiodirect import Thymio

In [2]:
# Set to True to use the Thymio simulator
use_simulator = True

# Sets the rate limiter to control the loop frequency in Hz
rate = RateLimiter(frequency=10.0) 

if use_simulator:
    port = 2001
    th = Thymio(use_tcp=True, host='localhost', tcp_port=port, on_connect=lambda node_id: print(f' Thymio {node_id} is connected'))
else:
    port = Connection.serial_default_port()
    th = Thymio(serial_port=port, on_connect=lambda node_id: print(f' Thymio {node_id} is connected'))

# Connect to Robot
th.connect()
robot = th[th.first_node()]

# Delay to allow robot initialization of all variables
time.sleep(1)

#TODO: 4.1 variables

state = "random"
ts = time.time()
ts_5_sec_counter = time.time()


 Thymio 1 is connected


In [4]:
def turn_robot(direction):
    if direction == "left":
        robot['motor.left.target'] = -100
        robot['motor.right.target'] = 100
    else:
        robot['motor.left.target'] = 100
        robot['motor.right.target'] = -100

def go_straight(forward):
    if forward:
        robot['motor.left.target'] = 100
        robot['motor.right.target'] = 100
    else:
        robot['motor.left.target'] = -100
        robot['motor.right.target'] = -100

def walk_straight_for_5_secs():
    pass


In [6]:
# don't delete the try-finally block, add your code inside the try block
try:
    # Main loop
    while True:
        
        #TODO: 4.2 collision avoidance
        if max(robot['prox.horizontal'][:5]) > 1000:
            turn_robot("right")
        #TODO: 4.3 random walk
        elif state == "random":
            if max(robot['prox.ground.reflected'][:2]) > 500:
                state = "follow"
            else:
                if ts_5_sec_counter > time.time() - 5:
                    go_straight(True)
                elif ts_5_sec_counter > time.time() - 6:
                    turn_robot(random.choice(["left", "right"]))
                else:
                    ts_5_sec_counter = time.time()
             
        #TODO: 4.4 line following
        elif state == "follow":
            pass
        else:
            go_straight(True)
        
        rate.sleep()
finally:
    robot['motor.left.target'] = 0
    robot['motor.right.target'] = 0
    print("Interrupted")

[loop_rate_limiters] [14:54:04] [warning] rate limiter is late by 572167.1 [ms] (rate_limiter.py:96)
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/ben/PycharmProjects/IAR2025/.venv/lib/python3.12/site-packages/thymiodirect/connection.py", line 69, in run
    msg = self.read_message()
          ^^^^^^^^^^^^^^^^^^^
  File "/home/ben/PycharmProjects/IAR2025/.venv/lib/python3.12/site-packages/thymiodirect/connection.py", line 62, in read_message
    raise error
  File "/home/ben/PycharmProjects/IAR2025/.venv/lib/python3.12/site-packages/thymiodirect/connection.py", line 54, in read_message
    payload_len = self.read_uint16()
                  ^^^^^^^^^^^^^^^^^^
  File "/home/ben/PycharmProjects/IAR2025/.venv/lib/python3.12/site-packages/thymiodirect/connection.py", line 39, in read_uint16
    b = self.io.read(2)
        ^^^^^^^^^^^^^^^
  File "/home/ben/PycharmProjects

BrokenPipeError: [Errno 32] Broken pipe